### Import required libraries

In [2]:
import numpy as np
import re

### Read the dataset

In [3]:
filename = 'Project (Application 1) (MetuSabanci Treebank).conll'

sentences = []
with open(filename, encoding="utf-8-sig") as infh:
    sentence = []
    for line in infh:
        if len(line) == 1:
            sentences.append(sentence.copy())
            sentence = []
        else:
            line_split = line.strip().split('\t')
            if line_split[1] != '_':
                if line_split[1].lower() == 'satın': # An error in the corpus that should be fixed by using POS 'Noun'
                    sentence.append([line_split[1], 'Noun'])
                else:
                    sentence.append([line_split[1], line_split[3]])

### Split the data into training and test sets

In [4]:
np.random.shuffle(sentences)
train_size = round(len(sentences)*0.9)
training = sentences[:train_size]
test = sentences[train_size:]

### Train the model

In [40]:
# Train seperate classifiers for the following features
# a non-first word whose first letter is capital (e.g. Ahmet -> Noun)
def non_first_capital_word_checker(word, **kwargs):
    return 'first_token' in kwargs and not kwargs['first_token'] and word[0] == word[0].upper()
# ends with -ıp, -ip, -up etc. (e.g. kurtulup, yapıp, edip etc. -> adj)
def endswith_up_checker(word, **kwargs):
    return re.search(r'[ı,i,u,ü]p$', word) is not None
# ends with -rdı, -rdi, -rdik, -rdim etc. (e.g. yapardı, koşardı, getirdik, ettim etc. -> verb)
def endswith_rdik_checker(word, **kwargs):
    return re.search(r'rd[ı,i,u,ü][m,k]*$', word) is not None
# ends with -yı, -yi etc. (e.g. yapmayı, götürüyü etc. -> noun)
def endswith_yi_checker(word, **kwargs):
    return re.search(r'y[ı,i,u,ü]$', word) is not None
# ends with -mış, -miş, -mışsınız, -mişiz etc. (e.g. yapmışız, koşmuşuz etc. -> verb)
def endswith_mis_checker(word, **kwargs):
    return re.search(r'm[ı,i,u,ü]ş[s]*[ı,i,u,ü]*[n]*[ı,i,u,ü]*[z]*$', word) is not None
# ends with -dı, -di, -diler, -tünüz etc. (e.g. yaptık, koştular etc. -> verb)
def endswith_di_checker(word, **kwargs):
    return re.search(r'[y]*[d,t][ı,i,u,ü][l]*[e,a]*[r]*[k]*[n]*[ı,i,u,ü]*[z]*[m]*$', word) is not None
# ends with -ca, -ce etc. (e.g. kısaca, kabaca etc. -> adv)
def endswith_ca_checker(word, **kwargs):
    return re.search(r'[c,ç][a,e]$', word) is not None
# ends with -ecek, -acak etc. (e.g. edeceğiz, yapacaklar etc. -> adv)
def endswith_acak_checker(word, **kwargs):
    return re.search(r'[a,e]c[a,e][ğ,k][s]*[l]*[a,e,ı,i]*[r]*[n]*[ı,i]*[z]*[m]*$', word) is not None
# ends with -an, -en etc. (e.g. yapan, edenleri etc. -> adv)
def endswith_an_checker(word, **kwargs):
    return re.search(r'[a,e]n[l]*[a,e]*[r]*[ı,i]*$', word) is not None
# ends with -e, -a, -de, -da, -den, -ten etc.
def endswith_a_checker(word, **kwargs):
    return re.search(r'[l]*[a,e]*[r]*[ı,i,u,ü][yl]*[m,n]*[ı,i,u,ü]*[z]*[d,t]*[a,e]*[n]*$', word) is not None
# ends with -lerine, -mize etc. (e.g. kendimize, kendilerine etc. -> adv)
def endswith_ine_checker(word, **kwargs):
    return re.search(r'[l]*[a,e]*[r]*[ı,i]*[m,n][a,e,ı,i][z]*[a,e]*$', word) is not None
# ends with -şme, -şma etc. (e.g. sürtüşme, kapışma etc. -> noun)
def endswith_sme_checker(word, **kwargs):
    return re.search(r'şm[a,e][l]*[a,e]*[r]*[ı,i]*[n]*$', word) is not None

features = [non_first_capital_word_checker, endswith_up_checker, endswith_rdik_checker, endswith_yi_checker, 
            endswith_mis_checker, endswith_di_checker, endswith_ca_checker, endswith_acak_checker, endswith_an_checker,
            endswith_a_checker, endswith_ine_checker, endswith_sme_checker]

In [41]:
# Extract the possible POS tags and Tokens in the training set into two sets
POS_tags = set()
tokens = set()
for sentence in training:
    for token, tag in sentence:
        POS_tags.add(tag)
        tokens.add(token)
        
# Convert tag and token sets into lists to have reference indices per tag/token
POS_tags = list(POS_tags)
tokens = list(tokens)

# Create and fill the transition and observation probability matrices
transition_probs = np.zeros((len(POS_tags)+1, len(POS_tags))) # +1 represents <s> (sentence beginning)
observation_probs = np.zeros((len(POS_tags), len(tokens)))
t_1_counts = np.zeros(len(POS_tags)+1)
t_counts = np.zeros(len(POS_tags))
feature_probs = np.zeros((len(POS_tags), len(features)))
unknown_probs = np.ones(len(POS_tags)) / 100 # TODO, NO SPECIFIC PRIORS YET!

for sentence in training:
    last_tag = '<s>'
    first_token = True
    for token, tag in sentence:
        t_1 = 0 if last_tag == '<s>' else POS_tags.index(last_tag) + 1
        t = POS_tags.index(tag)
        transition_probs[t_1, t] = transition_probs[t_1, t] + 1
        t_1_counts[t_1] = t_1_counts[t_1] + 1
        t_counts[t] = t_counts[t] + 1
        
        w = tokens.index(token)
        observation_probs[t, w] = observation_probs[t, w] + 1
        
        for feature in features:
            if feature(token, first_token=first_token):
                f = features.index(feature)
                feature_probs[t, f] = feature_probs[t, f] + 1
        
        last_tag = tag
        
        if first_token:
            first_token = False
        
transition_probs = (transition_probs.T/t_1_counts).T
observation_probs = (observation_probs.T/t_counts).T
feature_probs = (feature_probs.T/t_counts).T
unknown_probs = (unknown_probs.T/t_counts).T

### Test the model

Then, for the sentences in the test set, generate the most likely POS tag sequence using the
Viterbi algorithm. Devise a method for unknown words that is based on morphological and
orthographical information. Calculate the success rate of your tagger in terms of the number
of words correctly tagged. Also, calculate a sentence-based success rate, which is the ratio of
the number of correctly tagged sentences (i.e. all the words in the sentence are tagged
correctly) to the total number of sentences. In addition, compute the accuracy of each tag and
produce a confusion matrix.

In [42]:
def get_token_likelihood(state, token, b, first_token=False):
    if token in tokens:
        return b[state, tokens.index(token)]
    w = unknown_probs[state]
    for feature in features:
        if feature(token, first_token=first_token):
            f = features.index(feature)
            w = w * feature_probs[state, f]
    return w

def viterbi(states, observations, a, b):
    v = np.zeros((len(states)+2, len(observations)))
    backpointer = np.zeros((len(states)+2, len(observations))).astype(np.int)
    
    for s in range(1,len(states)+1):
        v[s,0] = a[0,s-1] * get_token_likelihood(s-1, observations[0], b, first_token=True)
        backpointer[s,0] = 0
    
    for t, o in enumerate(observations):
        if t == 0: continue
        for s in range(1,len(states)+1):
            v_max = -1
            v_argmax = -1
            bs = get_token_likelihood(s-1, o, b)
            for s_prime in range(1,len(states)+1):
                v_curr = v[s_prime,t-1] * a[s_prime,s-1] * bs
                if v_curr > v_max:
                    v_max = v_curr
                    v_argmax = s_prime
            v[s,t] = v_max
            backpointer[s,t] = v_argmax
    
    v[len(states)+1,len(observations)-1] = np.max(v[:,len(observations)-1])
    backpointer[len(states)+1,len(observations)-1] = max(1, np.argmax(v[:,len(observations)-1]))
    
    trace = []
    bp = backpointer[len(states)+1,len(observations)-1]
    row = len(states)+1
    column = len(observations)-1
    while row != 0:
        bp = backpointer[row, column]
        trace = [bp-1] + trace
        if row != len(states)+1:
            column = column - 1
        row = bp
    
    return [states[state_idx] for state_idx in trace[1:]]

In [43]:
viterbi(POS_tags, [token for token, tag in test[2]], transition_probs, observation_probs)

['Noun',
 'Noun',
 'Noun',
 'Noun',
 'Noun',
 'Noun',
 'Noun',
 'Punc',
 'Noun',
 'Postp',
 'Adv',
 'Noun',
 'Verb',
 'Punc']

In [44]:
test[2]

[['Rum', 'Adj'],
 ["Kesimi'nin", 'Noun'],
 ["AB'ye", 'Noun'],
 ['üye', 'Noun'],
 ['olması', 'Noun'],
 ['durumunda', 'Noun'],
 ['Türkiye', 'Noun'],
 ['-', 'Punc'],
 ['AB', 'Noun'],
 ['ilişkilerinde', 'Noun'],
 ['sürekli', 'Adv'],
 ['sürtüşme', 'Noun'],
 ['yaşanacak', 'Verb'],
 ['.', 'Punc']]

In [ ]:
correct = 0
tag_count = 0
for t in range(len(test)):
    pred = viterbi(POS_tags, [token for token, tag in test[t]], transition_probs, observation_probs)
    gold = list(map(lambda x: x[1], test[t]))
    correct += np.sum([1 if gold[i] == pred[i] else 0 for i in range(len(gold))])
    tag_count += len(gold)
print(correct, tag_count, correct/tag_count)

#### Prepare a design and implementation document which clearly explains the system. Follow the
structure given in http://www.cmpe.boun.edu.tr/~gungort/informationstudents.htm - Graduate
Courses “Programming Project Documentation”. Explain the modules, the data structures
used, the logic of the algorithms, etc. Show how the system learns the model parameters, the
training phase, and the executions on the test data. Include several example test sentences in
the document and show clearly the tagging process. The test cases should include
correct/incorrect tagging decisions, unknown words, etc. The system should be tested
throughly and the test scenarios should be included explicitly in the document.

The document will be an important part of the project. The suggested size of the document is
about 10-15 pages. Submit the document (with the source code as an appendix) both as hardcopy
and via Moodle; submit the program via Moodle. The dealine is 27.11.2018.
You will do a demonstration of the project. We will arrange for each group a date and hour
for the demo.
